<a id='top'>Top reference</a>
<h1 style="text-align:center">
<img src="https://m.gifmania.co.uk/TV-Series-Animated-Gifs/Animated-TV-channels-Logotypes/Netflix-Logo-91903.gif" style="width:900px;height:400px;">

### Notas de lançamento

- Versão 1: Análise exploratóra

- Versão 2: Modelo de recomendação

## Conteúdo

1. [Introdução](#1)
1. [Carregando os dados](#2)
1. [Visualização dos dados](#3)
1. [Modelo de Recomendação](#4)
1. [Conclusão](#5)
    
    

<a id="1"></a> <br>
# <div class="alert alert-block alert-info">Introducão</div>

**Programas de TV e filmes listados no Netflix
Este conjunto de dados consiste em programas de TV e filmes disponíveis no Netflix a partir de 2019. O conjunto de dados é coletado do Flixable, um mecanismo de busca Netflix de terceiros.**

**Em 2018, eles lançaram um relatório interessante que mostra que o número de programas de TV na Netflix quase triplicou desde 2010. O número de filmes do serviço de streaming diminuiu em mais de 2.000 títulos desde 2010, enquanto seu número de programas de TV quase triplicou. Será interessante explorar o que todos os outros insights podem ser obtidos no mesmo conjunto de dados.**

<a id="2"></a> <br>
# <div class="alert alert-block alert-info">Carregameno dos dados</div>

## Bibliotecas utilizadas

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import warnings

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots


In [ ]:
dados = pd.read_csv("/kaggle/input/netflix-shows/netflix_titles.csv")

dados.head()


# <div class="alert alert-danger">Pré processamento dos dados</div>

In [ ]:
dados.shape

In [ ]:
dados.dtypes

In [ ]:
dados.isnull().sum()

Devido a alta quantidade de dados faltantes na coluna *director* e *cast*, será removido apenas essas colunas para nao apagar 25% do dataset inicial. Além disso, o preenchimento dos valores nulos se tornam inviáveis. Pois essas informações necessitam de extração de dados de outro dataset que contenha o nome dos diretores e atores dos respectivos filmes.

In [ ]:
dados.drop(['director','cast'],axis = 1,inplace = True)

In [ ]:
dados.isnull().sum()/len(dados)*100

Ao observar o maior percentual (7,63%) dos valores faltantes em cada coluna, será aderido  a remoção de todos os registros faltantes da coluna *contry*, *date_added* e *ratting*.

In [ ]:
dados.dropna(inplace=True)

<h3 style="text-align:center;font-size:200%;;">Conclusão do projeto</h3>
<div class="progress">
  <div class="progress-bar" role="progressbar" style="width: 25%;" aria-valuenow="25" aria-valuemin="0" aria-valuemax="100">25% Concluído</div>
</div>

<a id="3"></a> <br>
# <div class="alert alert-block alert-info">Análise exploratória</div>

In [ ]:
plt.rcParams['figure.figsize'] = (13, 13)
wordcloud = WordCloud(stopwords=STOPWORDS,background_color = 'white', width = 800,  height = 800, max_words = 121).generate(' '.join(dados['description']))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Palavras mais populares na descrição da Netflix',fontsize = 40)


 
plt.imshow(wordcloud)

In [ ]:
values = dados["type"].value_counts()
labels = ["Filmes","TV Shows"]


fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent',
                             insidetextorientation='radial',marker_colors=["rgb(228,26,28)","rgb(102,102,102)"]
                            )])

fig.update_layout(height=600, width=800, title_text="Percentual de Filmes e TV Shows")
fig.show()

#### Após identificar o percentual refente aos Filmes (70,8%) e TV Shows (29,2%) no dataset. Foi então observado a quantidade de Filmes e TV Shows que foram lançados na plataforma da Netflix no mesmo ano que foi lançamento do Filme/TV Shows no cinema. 

In [ ]:
df_anos_movie = dados[dados["type"]== "Movie"]
df_anos_tv_shows = dados[dados["type"]== "TV Show"]
 
df_anos_movie.loc[:,["ano_lancamento"]] = 0
df_anos_tv_shows.loc[:,["ano_lancamento"]] = 0

warnings.filterwarnings("ignore")

In [ ]:
def ano_lancamento (df,data_adicionado):
    anos_lancamento_lista = []
    for x in data_adicionado:
        ano = int(x.split(", ")[1])
        anos_lancamento_lista.append(ano)
    df.loc[:,["ano_lancamento"]] = anos_lancamento_lista
    
ano_lancamento(df_anos_movie,df_anos_movie["date_added"])
ano_lancamento(df_anos_tv_shows,df_anos_tv_shows["date_added"])

In [ ]:
fig = go.Figure()

color_movie = ["rgb(251,180,174)",]*len(df_anos_movie["ano_lancamento"].unique())
lista_contagem_movie = list(df_anos_movie["ano_lancamento"].value_counts())
indice_movie = lista_contagem_movie.index(max(lista_contagem_movie))
color_movie[indice_movie] = "rgb(228,26,28)"


color_tv = ["rgb(179,179,179)"]*len(df_anos_tv_shows["ano_lancamento"].unique())
lista_contagem_tv = list(df_anos_tv_shows["ano_lancamento"].value_counts())
indice_tv = lista_contagem_tv.index(max(lista_contagem_tv))
color_tv[indice_tv] = "rgb(102,102,102)"

ano_tv = list(df_anos_tv_shows["ano_lancamento"].unique())
ano_movie = list(df_anos_movie["ano_lancamento"].unique())


fig.add_trace(go.Bar(x=df_anos_movie["ano_lancamento"].unique(),
                y=df_anos_movie["ano_lancamento"].value_counts(),
                name='Filves',
                marker_color=color_movie
                ))
fig.add_trace(go.Bar(x=df_anos_tv_shows["ano_lancamento"].unique(),
                y=df_anos_tv_shows["ano_lancamento"].value_counts(),
                name='TV Shows',
                marker_color=color_tv
                ))

fig.add_annotation(x=ano_movie[indice_movie], y=max(lista_contagem_movie),
            text="Maior entrada de Filmes",
            showarrow=True,
            arrowhead=1,
            font=dict(
               family="Courier New, monospace",
               size=16,
               color="#222A2A")
            )

fig.add_annotation(x=ano_tv[indice_tv], y=max(lista_contagem_tv),
            text="Maior entrada de TV Shows",
            showarrow=True,
            arrowhead=1,
            font=dict(
               family="Courier New, monospace",
               size=16,
               color="#222A2A")
            )


fig.update_layout(
    title='Quantidade de Filmes e TV Show adicionados a cada ano',
    xaxis_tickfont_size=14,
    xaxis=dict(
        title = "Ano de lançamento na Netflix",
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

In [ ]:
df_estreia_igual_movie = df_anos_movie[df_anos_movie["ano_lancamento"] == df_anos_movie["release_year"]].sort_index()
df_estreia_diferente_movie = df_anos_movie[df_anos_movie["ano_lancamento"] != df_anos_movie["release_year"]].sort_index()


df_estreia_igual_tv_shows = df_anos_tv_shows[df_anos_tv_shows["ano_lancamento"] == df_anos_tv_shows["release_year"]]
df_estreia_diferente_tv_shows = df_anos_tv_shows[df_anos_tv_shows["ano_lancamento"] != df_anos_tv_shows["release_year"]]

percentual_movie = len(df_estreia_igual_movie)*100 /(len(df_estreia_diferente_movie) + len(df_estreia_igual_movie))
percentual_tv = len(df_estreia_igual_tv_shows)*100 /(len(df_estreia_diferente_tv_shows) + len(df_estreia_igual_tv_shows))

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "pie"}, {"type": "pie"}]],subplot_titles=['Filmes', 'TV Shows'])


values = dados["type"].value_counts()
labels = ["Confirmados","Não confirmados"]


fig.add_trace(go.Pie(values=[percentual_movie,100 - percentual_movie],textinfo='label+percent',labels=labels,marker_colors=["rgb(228,26,28)","rgb(102,102,102)"]),row=1, col=1)
    

fig.add_trace(go.Pie(values=[percentual_tv,100 - percentual_tv],textinfo='label+percent',labels=labels,marker_colors=["rgb(228,26,28)","rgb(102,102,102)"]),row=1, col=2)

fig.update_layout(height=600, width=800, title_text="Percentual de Filmes e TV Shows lançados no mesmo ano de estreia")
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=1, subplot_titles=["Filmes","TV Shows"])


color_movie = ["rgb(228,26,28)"]*len(df_anos_movie["ano_lancamento"].unique())
list_movie = ["TV-MA","TV-14","TV-PG","R","NR","PG-13","PG","TV-Y7","TV-G","TV-Y","G","TV-Y7-FV","UR"]


color_tv = ["rgb(102,102,102)"]*len(df_anos_tv_shows["ano_lancamento"].unique())
list_tv_shows = ["TV-MA","TV-14","TV-PG","TV-Y7","TV-Y","TV-Y7-FV","TV-G","NR","R"]

fig.add_trace(go.Bar(x=list_movie,
                y=df_estreia_igual_movie["rating"].value_counts(),
                name='Filves',
                marker_color=color_movie
                ),row=1,col=1)

# ================= Percentual filmes ==============

per_filme_tv_14 = round(len(df_estreia_igual_movie[df_estreia_igual_movie["rating"]=="TV-14"])*100/(len(df_estreia_igual_movie)),2)
per_filme_tv_pg = round(len(df_estreia_igual_movie[df_estreia_igual_movie["rating"]=="TV-PG"])*100/(len(df_estreia_igual_movie)),2)
per_filme_tv_ma = round(len(df_estreia_igual_movie[df_estreia_igual_movie["rating"]=="TV-MA"])*100/(len(df_estreia_igual_movie)),2)

#================= linhas filmes =======================

fig.add_hline(y=len(df_estreia_igual_movie[df_estreia_igual_movie["rating"]=="TV-14"]),line_dash="dot", row=1, col=1,
              annotation_text=str(per_filme_tv_14)+"%", 
              annotation_position="bottom right")

fig.add_hline(y=len(df_estreia_igual_movie[df_estreia_igual_movie["rating"]=="TV-PG"]),line_dash="dot", row=1, col=1,
              annotation_text=str(per_filme_tv_pg)+"%",
              annotation_position="bottom right")

fig.add_hline(y=len(df_estreia_igual_movie[df_estreia_igual_movie["rating"]=="TV-MA"]),line_dash="dot", row=1, col=1,
              annotation_text=str(per_filme_tv_ma)+"%", 
              annotation_position="bottom right")

# ==================== Subplot 2 ========================

fig.add_trace(go.Bar(x=list_tv_shows,
                y=df_estreia_igual_tv_shows["rating"].value_counts(),
                name='TV Shows',
                marker_color=color_tv
                ),row=2,col=1)

# ================= Percentual tv shows ==============

per_tv_show_tv_14 = round(len(df_estreia_igual_tv_shows[df_estreia_igual_tv_shows["rating"]=="TV-14"])*100/(len(df_estreia_igual_tv_shows)),2)
per_tv_show_tv_pg = round(len(df_estreia_igual_tv_shows[df_estreia_igual_tv_shows["rating"]=="TV-PG"])*100/(len(df_estreia_igual_tv_shows)),2)
per_tv_show_tv_ma = round(len(df_estreia_igual_tv_shows[df_estreia_igual_tv_shows["rating"]=="TV-MA"])*100/(len(df_estreia_igual_tv_shows)),2)


#================= linhas tv shows =======================

fig.add_hline(y=len(df_estreia_igual_tv_shows[df_estreia_igual_tv_shows["rating"]=="TV-14"]),line_dash="dot", row=2, col=1,
              annotation_text=str(per_tv_show_tv_14)+"%", 
              annotation_position="bottom right")
fig.add_hline(y=len(df_estreia_igual_tv_shows[df_estreia_igual_tv_shows["rating"]=="TV-PG"]),line_dash="dot", row=2, col=1,
              annotation_text=str(per_tv_show_tv_pg)+"%", 
              annotation_position="bottom right")
fig.add_hline(y=len(df_estreia_igual_tv_shows[df_estreia_igual_tv_shows["rating"]=="TV-MA"]),line_dash="dot", row=2, col=1,
              annotation_text=str(per_tv_show_tv_ma)+"%",  
              annotation_position="bottom right")




fig.update_layout(
    title='Classificação para os filmes e tv shows ques estreiaram no mesmo ano de lançamento',
    xaxis_tickfont_size=14,
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1, # gap between bars of the same location coordinate.
    height=700, width=800
)



fig.show()

In [ ]:
paises = dados.country
lista_paises = []
for x in paises:
    separador = x.split(", ")
    for y in separador:
        lista_paises.append(y.lower())
        
dados_paises_lista = pd.DataFrame(lista_paises, columns = ["Contagem"])

contagem = dados_paises_lista["Contagem"].value_counts()
dados_paises = pd.DataFrame(contagem)
dados_paises.reset_index(inplace= True)

In [ ]:
codigo_paises= {'afghanistan': 'AFG',
 'albania': 'ALB',
 'algeria': 'DZA',
 'american samoa': 'ASM',
 'andorra': 'AND',
 'angola': 'AGO',
 'anguilla': 'AIA',
 'antigua and barbuda': 'ATG',
 'argentina': 'ARG',
 'armenia': 'ARM',
 'aruba': 'ABW',
 'australia': 'AUS',
 'austria': 'AUT',
 'azerbaijan': 'AZE',
 'bahamas': 'BHM',
 'bahrain': 'BHR',
 'bangladesh': 'BGD',
 'barbados': 'BRB',
 'belarus': 'BLR',
 'belgium': 'BEL',
 'belize': 'BLZ',
 'benin': 'BEN',
 'bermuda': 'BMU',
 'bhutan': 'BTN',
 'bolivia': 'BOL',
 'bosnia and herzegovina': 'BIH',
 'botswana': 'BWA',
 'brazil': 'BRA',
 'british virgin islands': 'VGB',
 'brunei': 'BRN',
 'bulgaria': 'BGR',
 'burkina faso': 'BFA',
 'burma': 'MMR',
 'burundi': 'BDI',
 'cabo verde': 'CPV',
 'cambodia': 'KHM',
 'cameroon': 'CMR',
 'canada': 'CAN',
 'cayman islands': 'CYM',
 'central african republic': 'CAF',
 'chad': 'TCD',
 'chile': 'CHL',
 'china': 'CHN',
 'colombia': 'COL',
 'comoros': 'COM',
 'congo democratic': 'COD',
 'Congo republic': 'COG',
 'cook islands': 'COK',
 'costa rica': 'CRI',
 "cote d'ivoire": 'CIV',
 'croatia': 'HRV',
 'cuba': 'CUB',
 'curacao': 'CUW',
 'cyprus': 'CYP',
 'czech republic': 'CZE',
 'denmark': 'DNK',
 'djibouti': 'DJI',
 'dominica': 'DMA',
 'dominican republic': 'DOM',
 'ecuador': 'ECU',
 'egypt': 'EGY',
 'el salvador': 'SLV',
 'equatorial guinea': 'GNQ',
 'eritrea': 'ERI',
 'estonia': 'EST',
 'ethiopia': 'ETH',
 'falkland islands': 'FLK',
 'faroe islands': 'FRO',
 'fiji': 'FJI',
 'finland': 'FIN',
 'france': 'FRA',
 'french polynesia': 'PYF',
 'gabon': 'GAB',
 'gambia, the': 'GMB',
 'georgia': 'GEO',
 'germany': 'DEU',
 'ghana': 'GHA',
 'gibraltar': 'GIB',
 'greece': 'GRC',
 'greenland': 'GRL',
 'grenada': 'GRD',
 'guam': 'GUM',
 'guatemala': 'GTM',
 'guernsey': 'GGY',
 'guinea-bissau': 'GNB',
 'guinea': 'GIN',
 'guyana': 'GUY',
 'haiti': 'HTI',
 'honduras': 'HND',
 'hong kong': 'HKG',
 'hungary': 'HUN',
 'iceland': 'ISL',
 'india': 'IND',
 'indonesia': 'IDN',
 'iran': 'IRN',
 'iraq': 'IRQ',
 'ireland': 'IRL',
 'isle of man': 'IMN',
 'israel': 'ISR',
 'italy': 'ITA',
 'jamaica': 'JAM',
 'japan': 'JPN',
 'jersey': 'JEY',
 'jordan': 'JOR',
 'kazakhstan': 'KAZ',
 'kenya': 'KEN',
 'kiribati': 'KIR',
 'north korea': 'PRK',
 'south korea': 'KOR',
 'kosovo': 'KSV',
 'kuwait': 'KWT',
 'kyrgyzstan': 'KGZ',
 'laos': 'LAO',
 'latvia': 'LVA',
 'lebanon': 'LBN',
 'lesotho': 'LSO',
 'liberia': 'LBR',
 'libya': 'LBY',
 'liechtenstein': 'LIE',
 'lithuania': 'LTU',
 'luxembourg': 'LUX',
 'macau': 'MAC',
 'macedonia': 'MKD',
 'madagascar': 'MDG',
 'malawi': 'MWI',
 'malaysia': 'MYS',
 'maldives': 'MDV',
 'mali': 'MLI',
 'malta': 'MLT',
 'marshall islands': 'MHL',
 'mauritania': 'MRT',
 'mauritius': 'MUS',
 'mexico': 'MEX',
 'micronesia': 'FSM',
 'moldova': 'MDA',
 'monaco': 'MCO',
 'mongolia': 'MNG',
 'montenegro': 'MNE',
 'morocco': 'MAR',
 'mozambique': 'MOZ',
 'namibia': 'NAM',
 'nepal': 'NPL',
 'netherlands': 'NLD',
 'new caledonia': 'NCL',
 'new zealand': 'NZL',
 'nicaragua': 'NIC',
 'nigeria': 'NGA',
 'niger': 'NER',
 'niue': 'NIU',
 'northern mariana islands': 'MNP',
 'norway': 'NOR',
 'oman': 'OMN',
 'pakistan': 'PAK',
 'palau': 'PLW',
 'panama': 'PAN',
 'papua new guinea': 'PNG',
 'paraguay': 'PRY',
 'peru': 'PER',
 'philippines': 'PHL',
 'poland': 'POL',
 'portugal': 'PRT',
 'puerto rico': 'PRI',
 'qatar': 'QAT',
 'romania': 'ROU',
 'russia': 'RUS',
 'rwanda': 'RWA',
 'saint kitts and nevis': 'KNA',
 'saint lucia': 'LCA',
 'saint martin': 'MAF',
 'saint pierre and miquelon': 'SPM',
 'saint vincent and the grenadines': 'VCT',
 'samoa': 'WSM',
 'san marino': 'SMR',
 'sao tome and principe': 'STP',
 'saudi arabia': 'SAU',
 'senegal': 'SEN',
 'serbia': 'SRB',
 'seychelles': 'SYC',
 'sierra leone': 'SLE',
 'singapore': 'SGP',
 'sint maarten': 'SXM',
 'slovakia': 'SVK',
 'slovenia': 'SVN',
 'solomon islands': 'SLB',
 'somalia': 'SOM',
 'south africa': 'ZAF',
 'south sudan': 'SSD',
 'spain': 'ESP',
 'sri lanka': 'LKA',
 'sudan': 'SDN',
 'suriname': 'SUR',
 'swaziland': 'SWZ',
 'sweden': 'SWE',
 'switzerland': 'CHE',
 'syria': 'SYR',
 'taiwan': 'TWN',
 'tajikistan': 'TJK',
 'tanzania': 'TZA',
 'thailand': 'THA',
 'timor-leste': 'TLS',
 'togo': 'TGO',
 'tonga': 'TON',
 'trinidad and tobago': 'TTO',
 'tunisia': 'TUN',
 'turkey': 'TUR',
 'turkmenistan': 'TKM',
 'tuvalu': 'TUV',
 'uganda': 'UGA',
 'ukraine': 'UKR',
 'united arab emirates': 'ARE',
 'united kingdom': 'GBR',
 'united states': 'USA',
 'uruguay': 'URY',
 'uzbekistan': 'UZB',
 'vanuatu': 'VUT',
 'venezuela': 'VEN',
 'vietnam': 'VNM',
 'virgin islands': 'VGB',
 'west bank': 'WBG',
 'yemen': 'YEM',
 'zambia': 'ZMB',
 'zimbabwe': 'ZWE'}

In [ ]:
abreviacao_paises = []
for x in dados_paises["index"]:
    if x in codigo_paises:
        abreviacao = codigo_paises[x]
        abreviacao_paises.append(abreviacao)
    else:
        abreviacao_paises.append(None)
    
dados_paises["Abreviacao"] = abreviacao_paises

In [ ]:
dados_paises.isnull().sum()

In [ ]:
dados_paises.dropna(inplace=True)

In [ ]:
fig = px.scatter_geo(dados_paises, locations=dados_paises["Abreviacao"], color="Contagem",
                     hover_name=dados_paises["index"].unique(), size=dados_paises["Contagem"],
                     projection="natural earth")

fig.update_geos ( 
    visible = True ,  resolution = 110 , 
    showcountries = True ,  countrycolor = "Black")

fig.update_layout(
    title='Paises com maior distribuição de Filmes e TV Shows',
    xaxis_tickfont_size=14)

fig.show()

In [ ]:
contador =0
top_paises = []
contagem_top = []
while contador < 11:
    top_paises.append(list(dados_paises.loc[contador,["index"]]))
    contagem_top.append(list(dados_paises.loc[contador,["Contagem"]]))
    contador = contador + 1
 

dados_top_paises = pd.DataFrame(top_paises, columns =["Paises"])
dados_top_paises_contagem = pd.DataFrame(contagem_top, columns=["Contagem"])


In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=dados_top_paises["Paises"],
                y=dados_top_paises_contagem["Contagem"],
                name='Filves',
                marker_color="rgb(228,26,28)"
                ))


fig.update_layout(
    title='Paises com as maiores quantidades de Filmes e TV Shows',
    xaxis_tickfont_size=14,
    
    
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1, # gap between bars of the same location coordinate.
    height=700, width=800
)



fig.show()

In [ ]:
# ================= Movie ==========================
generos_movie = []
for x in dados[dados["type"] == "Movie"]["listed_in"]:
    genero_movie = str(x.split(", ")[0])
    generos_movie.append(genero_movie)
    
    
dados_movie_contagem_genero = pd.DataFrame(generos_movie, columns = ["Contagem"])
contagem_movie = dados_movie_contagem_genero["Contagem"].value_counts()
dados_generos_movie = pd.DataFrame(contagem_movie)
dados_generos_movie.reset_index(inplace= True)

# ================ TV Shows =========================
generos_tv = []
for x in dados[dados["type"] == "TV Show"]["listed_in"]:
    genero_tv = str(x.split(", ")[0])
    generos_tv.append(genero_tv)
    
    
dados_tv_contagem_genero = pd.DataFrame(generos_tv, columns = ["Contagem"])
contagem_tv = dados_tv_contagem_genero["Contagem"].value_counts()
dados_generos_tv = pd.DataFrame(contagem_tv)
dados_generos_tv.reset_index(inplace= True)

In [ ]:
fig = make_subplots(rows=2, cols=1, subplot_titles=["Filmes","TV Shows"])

# ====================== Subplot 1 ======================
color_movie = ["rgb(228,26,28)"]*len(dados_generos_movie)
fig.add_trace(go.Bar(x=dados_generos_movie["index"],
                y=dados_generos_movie["Contagem"],
                name='Filves',
                marker_color=color_movie
                ),row=1,col=1)

# ================= Percentual + faixa =============

soma_filmes = int(dados_generos_movie.loc[0:4,["Contagem"]].sum())

percentual_top_filmes = round(soma_filmes*100/int(dados_generos_movie.loc[:,["Contagem"]].sum()),2)

fig.add_vrect(
    x0="Dramas", x1="Action & Adventure",
    fillcolor="rgb(251,180,174)", opacity=0.5,
    layer="below", line_width=0,row =1, col=1,
    annotation_text = str(percentual_top_filmes)+"% ",annotation_position = "outside top left"
)

# ==================== Subplot 2 ====================

color_tv = ["rgb(102,102,102)"]*len(dados_generos_tv)
fig.add_trace(go.Bar(x=dados_generos_tv["index"],
                y=dados_generos_tv["Contagem"],
                name='TV Shows',
                marker_color=color_tv
                ),row=2,col=1)

# ================= Percentual + faixa ==============

soma_tv = int(dados_generos_tv.loc[0:4,["Contagem"]].sum())

percentual_top_tv = round(soma_tv*100/int(dados_generos_tv.loc[:,["Contagem"]].sum()),2)

fig.add_vrect(
    x0="International TV Shows", x1="British TV Shows",
    fillcolor="rgb(179,179,179)", opacity=0.5,
    layer="below", line_width=0,row=2,col=1,
    annotation_text = str(percentual_top_tv)+"% ", annotation_position = "outside top left"
)

fig.update_layout(
    title='Gêneros de Filmes e TV Shows',
    xaxis_tickfont_size=14,
    barmode='group',
    bargap=0.4, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1, # gap between bars of the same location coordinate.
    height=800, width=900
)

fig.show()

<h3 style="text-align:center;font-size:200%;;">Conclusão do projeto</h3>
<div class="progress">
  <div class="progress-bar" role="progressbar" style="width: 50%;" aria-valuenow="25" aria-valuemin="0" aria-valuemax="100">50% Concluído</div>
</div>

<a id="4"></a> <br>
# <div class="alert alert-block alert-info">Modelo de Recomendação</div>

In [ ]:
dados_modelo = dados[["title","description"]]
dados_modelo.head()

# <div class="alert alert-warning">O que é TF-IDF?</div>

TF-IDF é um cálculo estatístico adotado pelo algoritmo do Google para medir quais termos são mais relevantes para um tópico, analisando a frequência com que aparecem em uma página, em comparação à sua frequência em um conjunto maior de páginas.

TF se refere à “frequência do termo”. Essa parte do cálculo responde à pergunta: com que frequência o termo aparece nesse documento? Quanto maior for a frequência no documento, maior será a importância do termo.

Já o IDF significa “frequência inversa dos documentos”. Nessa parte, a ferramenta responde: com que frequência o termo aparece em todos os documentos da coleção? Quanto maior for a frequência nos documentos, menor será a importância do termo.

Então, quando o fator IDF é incorporado, o cálculo diminui o peso dos termos que ocorrem com muita frequência no conjunto de documentos e aumenta o peso dos termos que ocorrem raramente. Este esquema ajuda a entender melhor:

![test](https://rockcontent.com/wp-content/uploads/2019/12/semrush.png)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Removendo palavras irrelevantes
tf_idf = TfidfVectorizer(stop_words='english')


#Construa a matriz TF-IDF necessária ajustando e transformando os dados
tfidf_matrix = tf_idf.fit_transform(dados_modelo['description'])

tfidf_matrix.shape

Existem 15491 palavras descritas para os 5743 filmes neste conjunto de dados.

Para determinar a pontuação,será utilizado a distância entre os grupos utilizando similaridade por cosseno.

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

#Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


indices = pd.Series(dados_modelo.index, index=dados_modelo['title']).drop_duplicates()

In [ ]:
def recomendacao(titulo, cosine_sim=cosine_sim):
    idx = indices[titulo]

    # Obtenha as pontuações de semelhança
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Classifique os filmes com base nas pontuações de similaridade
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtenha as pontuações dos 5 filmes mais semelhantes
    sim_scores = sim_scores[1:6]

    # Obtenha os índices de filmes
    movie_indices = [i[0] for i in sim_scores]

    # Retorne os 5 principais filmes mais semelhantes
    return dados_modelo['title'].iloc[movie_indices]

In [ ]:
recomendacao("Transformers Prime")

In [ ]:
recomendacao("Laddaland")

<h3 style="text-align:center;font-size:200%;;">Conclusão do projeto</h3>
<div class="progress">
  <div class="progress-bar" role="progressbar" style="width: 75%;" aria-valuenow="25" aria-valuemin="0" aria-valuemax="100">75% Concluído</div>
</div>

<a id="5"></a> <br>
# <div class="alert alert-success">Conclusão</div>

#### Podemos identificar que 71% dos dados desse dataset é composto por filmes e 29% composto por tv shows. Utilizando esse parâmetro para trilhar nosso caminho sobre a análise desses dados, foi visto que de todos os filmes e tv shows adicionados na Netflix, apenas 27% dos filmes e 48% dos tv shows foram adicionados no mesmo ano de estreia. Sendo que a frequência foi aumentando a partir de 2014, e chegando no ano de 2019 como sendo o ano com o maior percentual de filmes adicionados. Porém, em 2018 mostrou o maior percentual de tv shows adicionados na plataforma no mesmo ano de estreia.

#### Ao observar a classificação de idade permitida para todos os filmes e tv shows, foram observados que os filmes e tv shows apresentaram a mesma ordem de clássificação, TV-MA, TV-14 e TV-PG.

#### Os países com a maior quantidade de registros sobre a plataforma da Netflix são: 
1. Estados Unidos
2. India 
3. Reindo Unido
4. Canada
5. França 

#### Dentre todos os filmes no dataset, 82% são compostos por: 
1. Drama
2. Comédia 
3. Documentário
4. Ação e aventura


#### Dentre todos os tv shows no dataset, 75% são compostos por: 
1. TV Shows internacionais
2. TV Shows crimes 
3. Kids TV
4. TV Shows britânicos

#### Foi criado um modelo de recomendação utilizando o TD-IDF para determinação dos grupos de semelhança e para determinar a pontuação desses grupos foi utilizado a similaridade por cosseno. Por fim, é apresentado os 5 Filmes ou TV Shows que apresentam semelhaça na descrição do filme utilizado como busca no sistema de recomendação.


<h3 style="text-align:center;font-size:200%;;">Conclusão do projeto</h3>
<div class="progress">
  <div class="progress-bar" role="progressbar" style="width: 100%;" aria-valuenow="25" aria-valuemin="0" aria-valuemax="100">100% Concluído</div>
</div>

<a href="#top" class="btn btn-success btn-lg active" role="button" aria-pressed="true" style="color:white" data-toggle="popover" title="go to Colors">Voltar para o topo</a>